# 🐝 HornetRL: Development Environment Setup

Use this notebook to set up a persistent training environment on Google Colab.

**Features:**
* Mounts Google Drive (so your trained models are saved forever).
* Clones the repository safely (handles updates if it already exists).
* Installs dependencies in `editable` mode (great for tweaking code).
* Launches a training run on GPU.

In [ ]:
# 1. MOUNT GOOGLE DRIVE
from google.colab import drive
import os

print("--> Mounting Google Drive...")
drive.mount('/content/drive')

# 2. SETUP PROJECT DIRECTORY
# We creates a dedicated folder in your Drive to keep things organized
project_root = '/content/drive/MyDrive/HornetRL_Dev'

if not os.path.exists(project_root):
    print(f"--> Creating project folder at {project_root}...")
    os.makedirs(project_root)
else:
    print(f"--> Found existing project folder at {project_root}")

os.chdir(project_root)

In [ ]:
# 3. CLONE OR UPDATE REPOSITORY
repo_name = "hornetRL"

if not os.path.exists(repo_name):
    print("--> Cloning HornetRL repository...")
    !git clone https://github.com/lhooz/hornetRL.git
else:
    print("--> Repository already exists. Pulling latest changes...")
    os.chdir(repo_name)
    !git pull
    os.chdir('..') # Return to root to keep path consistent

In [ ]:
# 4. INSTALL DEPENDENCIES
import os

# Define the exact path to the inner repo folder
repo_path = os.path.join(project_root, repo_name)

# Move into the repo safely
try:
    os.chdir(repo_path)
    print(f"--> Working Directory: {os.getcwd()}")
except FileNotFoundError:
    print("Error: Repo folder not found. Did the clone step finish?")

# Install in editable mode (-e) so changes to .py files take effect immediately
print("--> Installing HornetRL package...")
!pip install -e .

In [ ]:
# 5. RUN TRAINING
# We save checkpoints to a specific folder inside your Drive repo

ckpt_dir = os.path.join(repo_path, "checkpoints_shac")
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

print(f"--> Starting Training on GPU...")
print(f"--> Saving checkpoints to: {ckpt_dir}")

# --gpu: Enables JAX GPU acceleration
# --dir: Specifies the output folder
!python -m hornetRL.train --gpu --dir "{ckpt_dir}"